# Lab 3

### Team Members:
 - Alex Lopez
 - Chris Haub
 - Erin McClure-Price
 - Chad Kwong

#### Library Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import turicreate as tc
import warnings


ModuleNotFoundError: No module named 'turicreate'

#### Data Import

In [ ]:
%%time

#Loading all data file separately 
vle = pd.read_csv('vle.csv')
assessments = pd.read_csv('assessments.csv')
courses = pd.read_csv('courses.csv')
studentAssessments = pd.read_csv('studentAssessment.csv')
studentInfo = pd.read_csv('studentInfo.csv')
studentRegistration = pd.read_csv('studentRegistration.csv')
studentVle = pd.read_csv('studentVle.csv')

In [ ]:
print('studentVle: \n',studentVle.head())
print('studentVle shape: ',studentVle.shape,'\n\n')

print('assessments: \n',assessments.head())
print('assessments shape: ', assessments.shape, '\n\n')

print('courses: \n',courses.head())
print('courses shape', courses.shape, '\n\n')

print('studentAssessments: \n', studentAssessments.head())
print('studentAssessments shape: ', studentAssessments.shape,'\n\n')

print('studentInfo: \n', studentInfo.head())
print('studentInfo shape', studentInfo.shape,'\n\n')

print('studentRegistration: \n', studentRegistration.head())
print('studentRegistration shape: ', studentRegistration.shape,'\n\n')

print('vle: \n', vle.head())
print('vle shape: ', vle.shape,'\n\n')

In [ ]:
%%time
dfFull = pd.merge(studentVle, studentInfo, on=['code_module', 'code_presentation', 'id_student'])
print(dfFull.shape)
dfFull.head()

# Business Understanding 1 #
*Describe the purpose of the data set you selected (i.e., why was this data collected in the first place?). How will you measure the effectiveness of a good algorithm? Why does your chosen validation method make sense for this specific dataset and the stakeholders needs?*

# Data Understanding 1 #
*Describe the meaning and type of data (scale, values, etc.) for each attribute in the data file. Verify data quality: Are there missing values? Duplicate data? Outliers? Are those mistakes? How do you deal with these problems?*

 - VLE stands for Virtual Learning Environment
 - VLE table records interactions with VLE 
 - 32,593 students
 - 22 courses
 

In [ ]:
%%time
# Randomly sampling 30,000 records StudentVle Table with Million Rows.
studentVleSample = studentVle.sample(n=35000, random_state = 777)
print(studentVleSample.shape)

# vle
# assessments
# courses
# studentAssessments
# studentInfo
# studentRegistration
# studentVleSample

df = pd.merge(studentVleSample, studentInfo, on=['code_module', 'code_presentation', 'id_student'])
df2 = pd.merge(studentAssessments, assessments, on='id_assessment')
df = pd.merge(df, df2, on=['code_module', 'code_presentation', 'id_student'])
print(df.shape)
df.head()

# Data Understanding 2 #
*Visualize the any important attributes appropriately. Important: Provide an interpretation for any charts or graphs.*

# Modeling and Evaluation 1 #
*Train and adjust parameters*

# Modeling and Evaluation 2	#
*Evaluate and Compare*

# Modeling and Evaluation 3 #
*Visualize Results*

# Modeling and Evaluation 4 #
*Summarize the Ramifications*

# Deployment #
*Be critical of your performance and tell the reader how you current model might be usable by other parties. Did you achieve your goals? If not, can you reign in the utility of your modeling? How useful is your model for interested parties (i.e., the companies or organizations that might want to use it)? How would your deploy your model for interested parties? What other data should be collected? How often would the model need to be updated, etc.?*

# Exceptional Work #